In [1]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [2]:
target = pd.read_csv('data/Ромбичность.csv', sep=';')
steel = pd.read_csv('data/Марка стали.csv', sep=';')
chemistry = pd.read_csv('data/Химия.csv', sep=';')
chemistry['AnalysisDate'] = pd.to_datetime(chemistry.AnalysisDate)
chemistry = chemistry.sort_values(by=['AnalysisDate'])

In [3]:
df_chem = chemistry.copy()

In [4]:
general_files = ['TundishWeight1', 'TundishWeight2', 'LadleSteelWgt', 'LadleTTE',
'ArmCastPos1', 'ArmCastPos2', 'TundishTemp1','TundishTemp2', 'LiquidusTemp', 'TundishTempDelta1',
'TundishTempDelta2', 'PercentMixSteel', 'BvoWaterTemp']

In [5]:
for item in tqdm(os.listdir('data')):
    if item not in ['Химия.csv', 'Ромбичность.csv', 'Марка стали.csv', 'Data.zip']:
        if any(file in item for file in general_files):
            df_tmp = pd.read_csv(f'data/{item}', sep=';', 
                                 names=['AnalysisDate', item.partition('#')[2][:-4]], header=0)
            df_tmp['AnalysisDate'] = pd.to_datetime(df_tmp.AnalysisDate, format='%d.%m.%Y %H:%M:%S')
            df_chem = pd.merge_asof(df_chem, 
                                    df_tmp.sort_values(by=['AnalysisDate']), 
                                    on='AnalysisDate', 
                                    direction="nearest")

100%|██████████| 281/281 [00:36<00:00,  7.69it/s]


In [6]:
dataset = pd.merge(target, steel, how='left', on='Heat')
dataset = pd.merge(dataset, df_chem, how='left', on='Heat')

In [7]:
dataset['AnalysisDate'] = dataset['AnalysisDate'].fillna(pd.to_datetime('1900-01-01 00:00:00'))

In [8]:
other_fields = ['StrStatus', 'StrOpenStreamCast', 'StrBilletNumber', 'StrEmergingLen', 'StrTotalCastingLength',
'StrMouldLevel', 'StrMouldLevelDeviation', 'StrStopperPos', 'StrStopperPosFiltered', 
'StrStopperPosFilteredDeviation', 'StrManualFlushing', 'StrCastingSpeed', 'StrOscillatorStroke',
'StrOscillatorFrequency', 'StrFriction', 'StrOilFlow', 'StrPowderFlow', 'StrMouldWaterFlow',
'StrMouldWaterPress', 'StrMouldWaterTemp', 'StrMouldWaterDeltaT', 'StrL1WaterFlow',
'StrL1WaterPressure', 'StrL2WaterFlow', 'StrL2WaterPressure', 'StrL3WaterFlow', 'StrL3WaterPressure',
'StrL4WaterFlow', 'StrL4WaterPressure', 'StrL5WaterFlow', 'StrL5WaterPressure', 'StrPR1Press', 'StrPR2Press']

In [9]:
dataset.shape

(38782, 46)

In [10]:
for FIELD in tqdm(other_fields):
    df_status = pd.DataFrame()
    for item in os.listdir('data'):
        if item not in ['Химия.csv', 'Ромбичность.csv', 'Марка стали.csv', 'Data.zip']:
            if FIELD in item:
                strand = item[-5:-4]
                df_tmp = pd.read_csv(f'data/{item}', sep=';', 
                                     names=['AnalysisDate', item.partition('#')[2][:-6]], header=0)
                df_tmp['AnalysisDate'] = pd.to_datetime(df_tmp.AnalysisDate, format='%d.%m.%Y %H:%M:%S')
                df_tmp['Strand'] = int(strand)
                df_status = df_status.append(df_tmp)

    dataset = pd.merge_asof(dataset.sort_values(by=['AnalysisDate']), 
                            df_status.sort_values(by=['AnalysisDate']), 
                            on='AnalysisDate', 
                            by='Strand',
                            direction="nearest")

 15%|█▌        | 5/33 [01:35<13:02, 27.96s/it]/home/anton/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
100%|██████████| 33/33 [27:45<00:00, 50.48s/it]


In [11]:
dataset.shape

(38782, 83)

In [12]:
dataset['AnalysisDate'] = np.where(dataset.AnalysisDate == '1900-01-01', 
                                   np.nan, 
                                   dataset.AnalysisDate == '1900-01-01')

In [15]:
col_list = list(dataset.iloc[:,16:].columns)

In [16]:
for col in tqdm(col_list):
    dataset[col] = dataset[col].str.replace('-','')
    dataset[col] = dataset[col].str.replace('<','')
    dataset[col] = dataset[col].str.replace('>','')
    dataset[col] = dataset[col].str.replace(',','.')
    dataset[col] = pd.to_numeric(dataset[col], errors='coerce')

100%|██████████| 67/67 [00:06<00:00,  9.72it/s]


In [17]:
dataset['AnalysisDate'] = pd.to_datetime(dataset.AnalysisDate)

In [18]:
dataset.to_csv('dataset_v2.csv', index=False)